<a href="https://colab.research.google.com/github/Noam-Coh3n/MLLM_practical/blob/main/Practical_Notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Notebook 1: Assessing Language Model Performance

## Please download to be able to save your answers in this notebook!

The practical is based around the paper [Marks and Tegmark (2023) The Geometry of Truth](https://arxiv.org/abs/2310.06824). In this paper, the authors examine the internal activations of large language models and find that true and false statements are linearly separable within the model.

In this first notebook, you will:

1. (0.5 pt) Assess the performance of a small model on one of Marks and Tegmark's datasets,
2. (1 pt) Examine how phrasing the prompt can make a difference to the performance of the model,
3. (2 pt) Evaluate the model on another dataset, and
4. (1.5 pt) Assess the performance of the model on this second dataset in comparison to a majority class baseline.

**Note**

In this notebook, we use the Llama-3.2 1B model. Before starting the tutorial, please go to the model's [huggingface page](https://huggingface.co/meta-llama/Llama-3.2-1B) and request permission to use the model.

## 0. Setup

In the practical, we will be using the package nnsight. In this notebook, we could technically get away without using it, but let's install and use it anyway, for practice.

In [ ]:
# This simply clears the outputs of a cell after running,
# so we don't see all the installation messages.
from IPython.display import clear_output

# Here we install the package nnsight
!pip install nnsight>0.5.0

clear_output()

When you signed up for model access, you probably obtained a [huggingface access token](https://huggingface.co/docs/hub/en/security-tokens). Here, you can use it to log in to this notebook.

In [ ]:
# Here, we log in to the huggingface hub with our access token
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We import some packages we will need, and set our random seeds (or try to!)
import random
import torch

random.seed(12)
torch.manual_seed(12)
torch.cuda.manual_seed(12)

## 1. Evaluating the model

LMs seem to be able to classify true and false statements in-context. Here, we will test how well that happens in a small model: Llama-3.2-1B.

We will be using [nnsight](https://nnsight.net/). nnsight allows us to run a model whilst keeping track of its internal states easily.

To run a model, we need to set up the model as an nnsight LanguageModel. The class LanguageModel acts as a wrapper for any model from the transformers library.

In [ ]:
# Here, we import the nnsight package, as well as a helper function clear_output
import nnsight

# We set up our chosen model as an nnsight LanguageModel. To change model, simply
# change the string "meta-llama/Llama-3.2-1B" to another huggingface model.
model = nnsight.LanguageModel("meta-llama/Llama-3.2-1B", device_map="auto")

clear_output()

# We can print out the structure of the model.
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

When we print out the model, we should get the following output.

This tells us that we have a LlamaModel with an Embedding layer going from a vocabulary of 128256 tokens to embeddings of dimension 2048.

We then have 16 LlamaDecoderLayers - aka Transformer blocks. Each of these consists of a self attention layer, an MLP, and two normalization components.

We then have another normalization component and the positional embeddings.

Finally, the component lm_head is a linear layer (aka a matrix) that goes from the 2048 dimensional embeddings to the 128256 dimensional vocabulary.

``` py
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2048, out_features=128256, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)
```

### Setting the Prompt Template.
A prompt is a sequence of words you give the language model for it to respond to. We are going to assess this language model on one of the datasets from Marks and Tegmark. In order to do so, we set up a prompt template.

In [ ]:
# We set up a prompt to see if models can correctly classify true and false statements.
CITIES_PROMPT_TEMPLATE = """The city of Tokyo is in Japan. This statement is: TRUE
The city of Hanoi is in Poland. This statement is: FALSE
{statement} This statement is:"""

source_statement = "The city of Toronto is in Canada." # true
source_prompt = CITIES_PROMPT_TEMPLATE.format(statement=source_statement)
base_statement = "The city of Chicago is in Canada." # false
base_prompt = CITIES_PROMPT_TEMPLATE.format(statement=base_statement)

# This should output FALSE if we gve it to the model.
print(base_prompt)

The city of Tokyo is in Japan. This statement is: TRUE
The city of Hanoi is in Poland. This statement is: FALSE
The city of Chicago is in Canada. This statement is:


### Downloading the dataset
We will now download the cities dataset from the paper, split it into train and test, and assess the model performance on the training set.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/saprmarks/geometry-of-truth/refs/heads/main/datasets/cities.csv')
print(f'This dataset has {len(df)} items')

# We can use train_test_split on a dataframe exactly as we are used to.
Xtrain, Xtest, ytrain, ytest = train_test_split(df.statement, df.label, test_size=0.2, random_state=12)
Xtrain.head()

This dataset has 1496 items


,statement
320,The city of Kakamega is in Kenya.
1122,The city of Jiaozuo is in China.
932,The city of Sofia is in Bulgaria.
511,The city of Kaifeng is in Mexico.
700,The city of Comilla is in Bangladesh.


### Assessing the model on the dataset
In the code below, we will evaluate the model on this dataset. Notice a few things. We use the context `with torch.no_grad():`. Since we are only evaluating the model, we do not need to save all the data from the forward pass that is needed for the backwards pass. We therefore tell the model not to save information needed for the gradients, i.e. `torch.no_grad()`.

We also use a context `with model.trace(base_prompt) as trace:` from nnsight that is not strictly necessary. When we run a model ith a trace, it means that we are able to access all the information occurring inside the model. This will be necessary in the next notebook, but we just introduce it here for practice.

We save the logits from the model output. Recall that the logits are the log of the probabilities of each token in the vocabulary (128256 tokens). We therefore want to take the highest value of the logits and return the index corresponding to this. We then decode this index into a text token.

In [ ]:
from tqdm import trange
import torch

predictions = []

with torch.no_grad():
  # We run the model over the dataset
  for i in trange(len(Xtrain)):
    statement = Xtrain.iloc[i]
    prompt = CITIES_PROMPT_TEMPLATE.format(statement=statement)
    with model.trace(prompt) as trace:
      # Recall that the logits are the log of the probabilities
      logits = model.output.logits.save()

    # Take the index of the highest logit and decode to a text token
    predictions.append(model.tokenizer.decode(logits.argmax(dim=-1)[0, -1]))



  0%|          | 0/1196 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

100%|██████████| 1196/1196 [03:17<00:00,  6.04it/s]


Let's look at the predictions: are they well formatted?


In [ ]:
print(set(predictions))

In [ ]:
numeric_predictions = [1 if p == ' TRUE' else 0 for p in predictions]
from sklearn.metrics import accuracy_score
acc = accuracy_score(ytrain, numeric_predictions)
print(f"The accuracy is: {acc}")

(0.5 pt) The accuracy is pretty low. Why is that? (Check the predictions)

**WRITE YOUR ANSWER HERE**

## 2. Phrasing the Prompt Differently
 Language models can be very sensitive to the way that they are prompted. Below, we will change the order of the true and false statements in the prompt template, and rerun.

In [ ]:
# Change the order of the true and false statements in the prompt template
CITIES_PROMPT_TEMPLATE = """The city of Hanoi is in Poland. This statement is: FALSE
The city of Tokyo is in Japan. This statement is: TRUE
{statement} This statement is:"""

(1 pt) Now,

a. Rerun the model on the dataset using this new prompt format and save the predictions (0.5 pt).

b. calculate the accuracy (0.5 pt).

In [ ]:
# Part 2a. (0.5 pt) Run the model over the new prompt format and save the predictions

In [ ]:
# Check whether the predictions are well formatted - not too chatty

In [ ]:
# Part 2b. (0.5 pt) Assess the accuracy of the predictions

## 3. Assessing the model on another dataset.
We have seen that with the right prompt, this model can tell the difference between true and false statements about cities. We will now look at another dataset from the paper.

The dataset `sp_en_trans.csv` is a dataset of correct and incorrect Spanish to English translations.

You will

a. (0.5 pt) Download the dataset `sp_en_trans.csv` from Marks and Tegmark and split into train and test.

b. (0.5 pt) Develop a new prompt template.

c. (0.5 pt) Assess the model on the train dataset, and

d. (0.5 pt) Calculate the accuracy of the model on the dataset.

**Note**: performance on this dataset is very dependent on the items that are in the prompt template. Do not worry about getting very good performance - this model does not do so well on this dataset. In particular, don't spend lots of time 'engineering' the prompt for better performance. You could try one or two different options, but that is not compulsory.

Once you have the output, you will check in part 4 whether the model is significantly better than a majority class baseline.

In [ ]:
# Part 3a.
# Download the dataset from github

# Split the dataset into train and test


In [ ]:
# Part 3b.
# Set up a prompt template for this model. Make sure the correct and incorrect
# examples are not in the dataset itself.


In [ ]:
# Part 3c.
# Run the model and save the predictions

In [ ]:
# Part 3d.
# Calculate the accuracy of the model

## 4. Testing against a majority baseline

**1.5 pt** Evaluate whether the accuracy of the model on the train set is significantly better than a majority baseline.

In [ ]:
# (0.5 pt) What is the accuracy of the majority baseline classifier?

# (1 pt) Choose a technique that will allow you to evaluate whether the model accuracy
# is significantly higher than the majority baseline, implement it,
# and assess whether the model performance is significantly higher than the baseline.